In [1]:
# Neural network for 
import pandas as pd
from text_cleaner import *
from tqdm import tqdm
import itertools
import torch
from torch.utils.data import DataLoader, Dataset

mapping = {
    "artist": 0,
    "caregiver": 1,
    "everyman": 2,
    "explorer": 3,
    "guru": 4,
    "hero": 5,
    "innocent": 6,
    "jester": 7,
    "magician": 8,
    "rebel": 9,
    "ruler": 10,
    "seducer": 11
}


In [2]:
# Load the Twitter dataset
twitter_df = pd.read_csv('tweets_14_03_2021.csv', index_col=0)

# Print the head of the loaded dataset
twitter_df.head()

# Clean-up the texts
twitter_df["cleaned_text"] = twitter_df["tweet_text"].apply(lambda x: clean_up_text(x))

# Tokenize the cleaned texts
twitter_df["cleaned_text"] = twitter_df["cleaned_text"].apply(lambda x: nltk.word_tokenize(x))

# Remove the stopwords
twitter_df["cleaned_text"] = twitter_df["cleaned_text"].apply(lambda x: remove_stopwords(x))

# Drop the rows with empty 'cleaned_text' field
twitter_df = twitter_df.drop(twitter_df[twitter_df['cleaned_text'].map(len) < 1].index)

# Print the new head of the dataset
twitter_df.head()

,_id,tweet_text,username,created_at,timestamp,archetype,cleaned_text
0,5f9f1c36b38e10f823bf2cdc,"@AndruEdwards The hard work has paid off, this...",LEGO_Group,2020-11-01 19:32:05.000,NaN,artist,"[hard, work, paid, awesome]"
1,5f9f1c36b38e10f823bf2cdd,@soosupersam A great way to surprise your love...,LEGO_Group,2020-11-01 19:09:40.000,NaN,artist,"[great, way, surprise, loved, one]"
2,5f9f1c36b38e10f823bf2cde,"You can now just bring the fun home, and reliv...",LEGO_Group,2020-11-01 14:00:36.000,NaN,artist,"[bring, fun, home, relive, favorite, childhood..."
3,5f9f1c36b38e10f823bf2cdf,@at_knb Happy birthday to the master builder! ...,LEGO_Group,2020-10-31 17:16:57.000,NaN,artist,"[happy, birthday, master, builder, hope, magic..."
6,5f9f1c36b38e10f823bf2ce2,@Ranchie This is the way! 😀,LEGO_Group,2020-10-31 15:16:26.000,NaN,artist,[way]


In [3]:
# Create full sentence out of the cleaned_text - detokenize
twitter_df["full_sentence"] = twitter_df["cleaned_text"].apply(lambda x: " ".join(x))

# Print the new head
twitter_df.head()

,_id,tweet_text,username,created_at,timestamp,archetype,cleaned_text,full_sentence
0,5f9f1c36b38e10f823bf2cdc,"@AndruEdwards The hard work has paid off, this...",LEGO_Group,2020-11-01 19:32:05.000,NaN,artist,"[hard, work, paid, awesome]",hard work paid awesome
1,5f9f1c36b38e10f823bf2cdd,@soosupersam A great way to surprise your love...,LEGO_Group,2020-11-01 19:09:40.000,NaN,artist,"[great, way, surprise, loved, one]",great way surprise loved one
2,5f9f1c36b38e10f823bf2cde,"You can now just bring the fun home, and reliv...",LEGO_Group,2020-11-01 14:00:36.000,NaN,artist,"[bring, fun, home, relive, favorite, childhood...",bring fun home relive favorite childhood memor...
3,5f9f1c36b38e10f823bf2cdf,@at_knb Happy birthday to the master builder! ...,LEGO_Group,2020-10-31 17:16:57.000,NaN,artist,"[happy, birthday, master, builder, hope, magic...",happy birthday master builder hope magical day
6,5f9f1c36b38e10f823bf2ce2,@Ranchie This is the way! 😀,LEGO_Group,2020-10-31 15:16:26.000,NaN,artist,[way],way


In [4]:
# Load PyTorch device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Convert labels to numerical
twitter_df["archetype_num"] = twitter_df["archetype"]

# Extract the crucial columns and print dataframe head
df = twitter_df[["archetype_num", "full_sentence"]]

# Replace string labels with numbers, drop NAN
df = df.replace({"archetype_num": mapping})
df = df.dropna()

df.head()

print(df["archetype_num"].unique())

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11.]


In [6]:
# Convert the dataset to a list of tuples, as iterator
class TextDataset(Dataset):
    def __init__(self, dataframe):
        self.text_list = dataframe.to_records(index=False)
        self.list_iterator = iter(list(self.text_list))
    
    def __len__(self):
        return len(self.text_list)
    
    def __getitem__(self, idx):
        return self.text_list[idx]

In [7]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab

# Tokenize the data
dataset_full = TextDataset(df)
tokenizer = get_tokenizer('basic_english')
counter = Counter()
for (label, line) in dataset_full:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

In [8]:
# Define processing pipelines
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x)

In [9]:
# Collation function and create a DataLoader object
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

dataset_full = TextDataset(df)
data_loader = DataLoader(dataset_full, batch_size=16, shuffle=False, collate_fn=collate_batch)

In [10]:
# Model definition
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [11]:
# Instantiate the model
dataset_full = TextDataset(df)
num_class = len(set([label for (label, text) in dataset_full]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [12]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [13]:
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 32 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
dataset_iter = TextDataset(df)
train_dataset = list(dataset_iter)
num_train = int(len(train_dataset) * 0.8)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

num_train = int(len(split_train_) * 0.75)

split_train_, test_dataset = \
    random_split(split_train_, [num_train, len(split_train_) - num_train])
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 2085 batches | accuracy    0.304
| epoch   1 |  1000/ 2085 batches | accuracy    0.462
| epoch   1 |  1500/ 2085 batches | accuracy    0.522
| epoch   1 |  2000/ 2085 batches | accuracy    0.557
-----------------------------------------------------------
| end of epoch   1 | time:  4.62s | valid accuracy    0.485 
-----------------------------------------------------------
| epoch   2 |   500/ 2085 batches | accuracy    0.638
| epoch   2 |  1000/ 2085 batches | accuracy    0.633
| epoch   2 |  1500/ 2085 batches | accuracy    0.640
| epoch   2 |  2000/ 2085 batches | accuracy    0.652
-----------------------------------------------------------
| end of epoch   2 | time:  4.30s | valid accuracy    0.506 
-----------------------------------------------------------
| epoch   3 |   500/ 2085 batches | accuracy    0.710
| epoch   3 |  1000/ 2085 batches | accuracy    0.689
| epoch   3 |  1500/ 2085 batches | accuracy    0.698
| epoch   3 |  2000/ 2085 batches | accuracy

In [14]:
# Evaluate the model
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.669
